## 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
import pandas as pd 
from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 讀取資料
cancer = datasets.load_breast_cancer()
df_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
target = pd.DataFrame(cancer.target, columns=["Target"])
df = pd.concat([df_cancer,target], axis=1)
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# 切分訓練集/測試集
X_train, X_test, y_train, y_test = train_test_split(df_cancer, target, test_size=0.3,  random_state=10)

# 建立模型
gbc = GradientBoostingClassifier()

# 訓練模型
gbc.fit(X_train, y_train)

# 預測測試集
y_pred = gbc.predict(X_test)

# 預測值與實際值的準確度，使用 accuracy score
print("Accuracy: %.3f" % accuracy_score(y_test, y_pred))

Accuracy: 0.965


In [4]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150, 200, 250]
max_depth = [1, 3, 5, 7, 9]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(gbc, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %.3f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.970 using {'max_depth': 3, 'n_estimators': 250}


In [6]:
# 使用最佳參數重新建立模型
gbc_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_["max_depth"],
                                           n_estimators=grid_result.best_params_["n_estimators"])

# 訓練模型
gbc_bestparam.fit(X_train, y_train)

# 預測測試集
y_pred = gbc_bestparam.predict(X_test)

# 調整參數後的 accuracy
print("Accuracy: %.3f" % accuracy_score(y_test, y_pred))

Accuracy: 0.971
